In [1]:
import numpy as np
import pandas as pd
import matlab.engine
import yaml
import time
import importlib
import pathlib
import matplotlib.pyplot as plt
import csv
from tqdm import tqdm
from scipy.stats import pearsonr
from loguru import logger
from functools import partial, wraps
from itertools import product
import os
from numbers import Number

In [2]:
def BF_zscore(input_data):
    """
    Z-score the input data vector.

    Parameters:
    input_data (array-like): The input time series (or any vector).

    Returns:
    numpy.ndarray: The z-scored transformation of the input.

    Raises:
    ValueError: If input_data contains NaN values.
    """
    # Convert input to numpy array
    input_data = np.array(input_data)

    # Check for NaNs
    if np.isnan(input_data).any():
        raise ValueError('input_data contains NaNs')

    # Z-score twice to reduce numerical error
    zscored_data = (input_data - np.mean(input_data)) / np.std(input_data, ddof=1)
    zscored_data = (zscored_data - np.mean(zscored_data)) / np.std(zscored_data, ddof=1)

    return zscored_data


In [3]:
eng = matlab.engine.start_matlab()

In [4]:
proj_root = pathlib.Path("/Users/jmoo2880/Documents/hctsa")
eng.addpath(eng.genpath(str(proj_root)), nargout=0)

In [5]:
def zscore_decorator(func):
    @wraps(func)
    def wrapper(y, *args, **kwargs):
        y = BF_zscore(y)
        return func(y, *args, **kwargs)
    return wrapper

def range_constructor(loader, node):
    start, end = loader.construct_sequence(node)
    return list(range(start, end+1))
yaml.add_constructor("!range", range_constructor)

In [6]:
def load_yaml(file):
    print(f"Loading configuration file: {file.split('/')[-1]}")
    funcs = {}
    with open(file) as f:
        yf = yaml.load(f, Loader=yaml.FullLoader)

    for module_name in yf:
        print(f"\n*** Importing module {module_name} *** \n")
        module = importlib.import_module(module_name)
        for function_name in yf[module_name]:
            # Get the function's configuration dictionary
            function_config = yf[module_name][function_name]
            # If no configs section exists or if it's empty, use a list with single empty dict
            if ('configs' not in function_config or function_config.get('configs') is None or 
                function_config.get('configs') == []):
                configs = [{}]
            else:
                configs = function_config.get('configs', [{}])

            for params in configs:
                # Handle the case where params is None
                if params is None:
                    params = {}
                    
                zscore_first = params.pop("zscore", False)
                param_keys, param_vals = zip(*params.items()) if params else ([], [])
                
                param_combinations = [dict(zip(param_keys, values)) 
                                   for values in product(*[v if isinstance(v, list) 
                                                        else [v] for v in param_vals])]
                
                # If no parameter combinations were generated, add empty dict
                if not param_combinations:
                    param_combinations = [{}]
                
                # create a function for each parameter combination
                for param_set in param_combinations:
                    feature_name = (f"{module_name}_{function_name}_" + 
                                  "_".join(f"{v}" for k, v in param_set.items())
                                  if param_set else f"{module_name}_{function_name}")
                    if not zscore_first:
                        feature_name += "_raw"
                    
                    print(f"Adding operation {feature_name} with params {param_set} "
                          f"(Z-score={zscore_first})")
                    
                    base_func = partial(getattr(module, function_name), **param_set)
                    if zscore_first:
                        base_func = zscore_decorator(base_func)
                    
                    # return the MATLAB callable corresponding to the python implementation for direct comparison
                    # make sure to check whethe the data needs to be zscored when calling the MATLAB func, cannot be wrapped as it is not a python function
                    # so needs to be done manually when calling the function.
                    hctsa_name = function_config.get('hctsa_name')
                    hctsa_callable = eval(f"eng.{hctsa_name}")

                    # keep ordered args only for testing YAML otherwise bloats
                    funcs[feature_name] = {'callable': base_func, 'params': param_set, 'hctsa_name': function_config.get('hctsa_name'), 
                                           'matlab_callable': hctsa_callable, 'isZscore': zscore_first, 'ordered_args': function_config.get('ordered_args')}
                    
    return funcs

In [7]:
funcs = load_yaml("/Users/jmoo2880/Documents/py-hctsa-project/pyhctsa/Configurations/criticality.yaml")

Loading configuration file: criticality.yaml

*** Importing module Criticality *** 

Adding operation Criticality_RAD_1 with params {'tau': 1} (Z-score=True)
Adding operation Criticality_RAD_2 with params {'tau': 2} (Z-score=True)
Adding operation Criticality_RAD_tau with params {'tau': 'tau'} (Z-score=True)


In [8]:
empirical1000 = []
with open("../../../empirical1000/hctsa_timeseries-data.csv") as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        # Convert each element from string to float (or int if appropriate)
        try:
            time_series = [float(value) for value in row if value != '']
            empirical1000.append(time_series)
        except ValueError as e:
            print(f"Skipping row due to conversion error: {row}")
            continue

In [9]:
def eval_comparison(yaml, data):
    func_dict = load_yaml(yaml)
    func_res = dict()
    for func in func_dict:
        print(f"Evalutating {func}")
        f = func_dict[func]
        python_func = f['callable']
        matlab_func = f['matlab_callable']
        hctsa_name = f['hctsa_name']
        isZscore = f['isZscore']
        params = f['params']
        ordered_args = []
        if params:
            order = f['ordered_args']
            ordered_args = [params[k] for k in order]
            
        print(f"Comparing to {hctsa_name}")
        res = dict()
        for i in range(len(data)):
            x = data[i]
            matlab_eval = matlab_func(BF_zscore(matlab.double(x)), *ordered_args) if isZscore else matlab_func(matlab.double(x), *ordered_args)
            python_eval = python_func(x)
            res[i] = {'matlab': matlab_eval, 'python': python_eval}
        func_res[func] = res
    return func_res

In [13]:
out = eval_comparison("/Users/jmoo2880/Documents/py-hctsa-project/pyhctsa/Configurations/criticality.yaml", empirical1000)

Loading configuration file: criticality.yaml

*** Importing module Criticality *** 

Adding operation Criticality_RAD_1 with params {'tau': 1} (Z-score=True)
Adding operation Criticality_RAD_2 with params {'tau': 2} (Z-score=True)
Adding operation Criticality_RAD_tau with params {'tau': 'tau'} (Z-score=True)
Evalutating Criticality_RAD_1
Comparing to CR_RAD

sigma_dx =

    0.9131

0.6147442178582068
0.19397150795009535
0.49630454655667594
0.1983454036404054
0.4167212032367406
0.21056486501273491
0.4462390986662703
0.2059569625625757
0.4482167941829026
0.2019043584301299
0.5227624318383141
0.20124255005096545
1.04139658163176
0.09610739681049035
0.9129484036785128
0.12830514482362904
0.811366974670785
0.15908862084446976
0.4416024238762775
0.0
0.6028274907702723
0.0
0.6948081756835145
0.16863539384185738
0.909782664261281
0.13699656201389074
0.6678687215696919
0.169588664094011
1.0957924809126975
0.10132476142287457
0.35673833254636933
0.0
0.3833831192747302
0.0
0.5097293545424673
0.19

/opt/anaconda3/envs/pyhctsa/lib/python3.12/site-packages/numpy/_core/_methods.py:227: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/opt/anaconda3/envs/pyhctsa/lib/python3.12/site-packages/numpy/_core/_methods.py:184: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/opt/anaconda3/envs/pyhctsa/lib/python3.12/site-packages/numpy/_core/_methods.py:219: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


0.8046926179252947
0.10016852996716093
0.6002318643116469
0.15975078299044346
0.9654803393304998
0.0947267206082962
0.6341792398788816
0.1678828647361987
0.7207247680021558
0.1284343173286351
0.6318432710118463
0.12179536120161331
0.8024004695655123
0.11143142908358666
0.8466710405297124
0.10437556033896349
0.804101973233711
0.08510744084713308
0.5492343072161208
0.18424552872152578
1.0775210703301707
0.05991069732632988
0.5214224284555712
0.18366980576958034
0.7153583463818414
0.165648000169656
0.7166607192034758
0.1655127295169859
1.0999797181824758
0.09177520045400238
1.2705717232463576
0.054082187632917264
0.8060147329444671
0.1471619240733141
0.7656657703060733
0.1501377548423366
0.7297069467615858
0.1283294221037696
0.6376537094655188
0.15652682625297443
0.49781292840760605
0.1883856488050125
0.642654359657351
0.17398785257183527
0.7376086265894158
0.1409944635455299
0.5502685121862169
0.17584181450258848
0.47909381519852307
0.18772169703705094
0.49683591370912455
0.1899067871341

In [15]:
def compare_outputs(outputs):
    """
    Compare MATLAB and Python feature outputs, computing relative errors
    and Pearson correlations.

    Parameters
    ----------
    outputs : dict
        Nested dictionary of feature values with structure:
        {
            feature_name: {
                ts_id: {
                    'matlab': scalar_or_dict,
                    'python': scalar_or_dict
                }
            }
        }

    Returns
    -------
    results : dict
        Dictionary mapping feature (or subfeature) names to correlation and stats:
        {
            'feature.subkey': {
                'r': float,
                'pval': float,
                'res_py': ndarray,
                'res_matlab': ndarray
            }
        }
    """
    flat = {}
    for feat, ts_dict in outputs.items():
        for ts, run in ts_dict.items():
            ml = run['matlab']
            py = run['python']
            if isinstance(ml, dict) and isinstance(py, dict):
                for k, mlv in ml.items():
                    pyv = py[k]
                    slot = f"{feat}.{k}"
                    flat.setdefault(slot, {})[ts] = (mlv, pyv)
            elif isinstance(ml, Number) and isinstance(py, Number):
                flat.setdefault(feat, {})[ts] = (ml, py)
            else:
                raise ValueError(f"Feature {feat}@{ts} is neither both scalars nor both dicts.")

    results = {}
    for slot, tsmap in flat.items():
        ml_vals, py_vals = [], []
        for ts, (mlv, pyv) in tsmap.items():
            ml_vals.append(mlv)
            py_vals.append(pyv)
            
            both_finite = np.isfinite(mlv) and np.isfinite(pyv)
            both_nan = np.isnan(mlv) and np.isnan(pyv)
            both_posinf = (mlv == np.inf) and (pyv == np.inf)
            both_neginf = (mlv == -np.inf) and (pyv == -np.inf)

            if both_finite:
                if mlv == 0:
                    rel_err = np.nan
                else:
                    rel_err = abs(mlv - pyv) / abs(mlv) * 100
                print(f"[{slot} | ts={ts}]  RelErr% = {rel_err:.2f}")
            elif both_nan or both_posinf or both_neginf:
                print(f"[{slot} | ts={ts}]  RelErr% = MATCH (both non-finite)")
            else:
                print(f"[{slot} | ts={ts}]  RelErr% = NaN (mismatch in finiteness)")

        ml_arr = np.array(ml_vals, dtype=float)
        py_arr = np.array(py_vals, dtype=float)
        finite_mask = np.isfinite(ml_arr) & np.isfinite(py_arr)

        if finite_mask.sum() > 1 and ml_arr[finite_mask].std() and py_arr[finite_mask].std():
            r, p = pearsonr(ml_arr[finite_mask], py_arr[finite_mask])
        else:
            r, p = np.nan, np.nan

        results[slot] = {
            'r': r,
            'pval': p,
            'res_py': py_arr,
            'res_matlab': ml_arr
        }

    return results

In [16]:
results = compare_outputs(out)

[Criticality_RAD_1 | ts=0]  RelErr% = 0.00
[Criticality_RAD_1 | ts=1]  RelErr% = 0.00
[Criticality_RAD_1 | ts=2]  RelErr% = 0.00
[Criticality_RAD_1 | ts=3]  RelErr% = 0.00
[Criticality_RAD_1 | ts=4]  RelErr% = 0.00
[Criticality_RAD_1 | ts=5]  RelErr% = 0.00
[Criticality_RAD_1 | ts=6]  RelErr% = 0.00
[Criticality_RAD_1 | ts=7]  RelErr% = 0.00
[Criticality_RAD_1 | ts=8]  RelErr% = 0.00
[Criticality_RAD_1 | ts=9]  RelErr% = MATCH (both non-finite)
[Criticality_RAD_1 | ts=10]  RelErr% = MATCH (both non-finite)
[Criticality_RAD_1 | ts=11]  RelErr% = 0.00
[Criticality_RAD_1 | ts=12]  RelErr% = 0.00
[Criticality_RAD_1 | ts=13]  RelErr% = 0.00
[Criticality_RAD_1 | ts=14]  RelErr% = 0.00
[Criticality_RAD_1 | ts=15]  RelErr% = MATCH (both non-finite)
[Criticality_RAD_1 | ts=16]  RelErr% = MATCH (both non-finite)
[Criticality_RAD_1 | ts=17]  RelErr% = 0.00
[Criticality_RAD_1 | ts=18]  RelErr% = 0.00
[Criticality_RAD_1 | ts=19]  RelErr% = 0.00
[Criticality_RAD_1 | ts=20]  RelErr% = 0.00
[Criticali

Non-matching outputs due to MATLAB division by near-zero, but finite value and python rounding to zero, giving an inf

In [43]:
mask_py = np.isfinite(results['Criticality_RAD_1']['res_py'])
mask_ml = np.isfinite(results['Criticality_RAD_1']['res_matlab'])

In [51]:
pearsonr(results['Criticality_RAD_1']['res_py'][mask_py], results['Criticality_RAD_1']['res_matlab'][mask_ml])

PearsonRResult(statistic=np.float64(-0.0010131712259358896), pvalue=np.float64(0.9746266305486396))

In [49]:
np.mean(results['Criticality_RAD_1']['res_py'][mask_py])

np.float64(-134602099160264.56)

In [17]:
eng.CR_RAD(matlab.double(BF_zscore(empirical1000[601])), 1)

-inf

In [11]:
from Criticality import RAD

In [18]:
RAD(BF_zscore(empirical1000[601]))

1.3283286473948586
6.964931120730528e-18


np.float64(-1.3298687397033098e+17)